In [31]:
def simulate(price_history): 
    diffs = np.diff(np.log(price_history))

    std = np.std(diffs) ** 2
    drift = np.mean(diffs) + std ** 2 / 2

    future = []
    for _ in range(1): 
        future.append(np.cumsum(np.random.normal(drift, std, 3)))

    future_prices_estimates = price_history[-1] * np.exp(np.array(future))

    return np.mean(future_prices_estimates), np.std(future_prices_estimates)    



In [33]:
simulate(price_history)

(83.43427391502001, 0.5352531132215701)

In [2]:
from time import time
import cProfile


start = time()

from simulation import Simulation
from maker import SimpleMarketMaker as MarketMaker

print("Welcome to the game!")
mm = MarketMaker()
sim = Simulation(mm)
sim.run(logging=True)
sim.summarize(logging=True)

end = time()

print(end - start)

Welcome to the game!
RUNNN
[100.5, 99.8980484008789, 100.16717529296875, 100.30915069580078, 100.60249328613281, 100.83250427246094, 100.99629974365234, 100.9830322265625, 100.9766616821289, 101.04842376708984] [99.5, 100.03199768066406, 100.26638793945312, 100.36935424804688, 100.68502044677734, 101.00607299804688, 101.16409301757812, 101.10124969482422, 101.07162475585938, 101.19403839111328]
[-0.00034983  0.0025153   0.00122131  0.00303015  0.00273395  0.00159315
 -0.00037656 -0.00017814  0.00096057]
1.6020001466213905e-06 0.001238879865023832
101.37211709983201 0.1025355852964647
RUNNN
[100.5, 99.8980484008789, 100.16717529296875, 100.30915069580078, 100.60249328613281, 100.83250427246094, 100.99629974365234, 100.9830322265625, 100.9766616821289, 101.04842376708984, 101.28092956542969] [99.5, 100.03199768066406, 100.26638793945312, 100.36935424804688, 100.68502044677734, 101.00607299804688, 101.16409301757812, 101.10124969482422, 101.07162475585938, 101.19403839111328, 101.52024078

In [1]:
from time import time
import cProfile


start = time()

from simulation import Simulation
from maker import SimpleMarketMaker as MarketMaker

print("Welcome to the game!")
mm = MarketMaker()
sim = Simulation(mm)
sim.run(logging=True)
sim.summarize(logging=True)

end = time()

print(end - start)

Welcome to the game!
Total profit: -9813.815948486328
Total holding: 0 at price 368.94427490234375 for a total of 0.0
Total cash: 186.18405151367188
Final Revenue: 186.18405151367188
2.1454601287841797


- Expect price reversion based off history  
- spread should be based off history 

In [5]:
import cProfile

data = cProfile.run('sim.run(logging=True)', 'restats_list.prof')

KeyboardInterrupt: 

In [ ]:
file = "restats_list.prof"

In [ ]:
import cProfile

data = cProfile.run('sim.run(logging=True)', file)

In [3]:
import pstats

p = pstats.Stats(file)
p.sort_stats(pstats.SortKey.TIME)
p.print_stats()

Sun Nov 17 14:14:54 2024    restats

         789802 function calls in 1371.441 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   256347 1365.365    0.005 1365.365    0.005 C:\Users\megat\Yash\yashcoding\Python Coding\ATC\MM-game-skeleton-main\logger.py:6(log)
        1    2.677    2.677 1371.441 1371.441 C:\Users\megat\Yash\yashcoding\Python Coding\ATC\MM-game-skeleton-main\simulation.py:193(run)
    23400    1.000    0.000  205.758    0.009 C:\Users\megat\Yash\yashcoding\Python Coding\ATC\MM-game-skeleton-main\simulation.py:99(executeLimitOrders)
    23400    0.945    0.000    0.945    0.000 {built-in method mm_game._core.getNextPrices}
    23400    0.284    0.000  150.146    0.006 C:\Users\megat\Yash\yashcoding\Python Coding\ATC\MM-game-skeleton-main\simulation.py:46(checkAndUpdate)
    23401    0.257    0.000  379.290    0.016 C:\Users\megat\Yash\yashcoding\Python Coding\ATC\MM-game-skeleton-main\logger.py:18(spacing)
